In [1]:
import numpy as np
import time
from sklearn.svm import LinearSVC
from collections import defaultdict
from sklearn.linear_model import LogisticRegression

In [46]:
train_data = np.loadtxt("train.dat")
test_data = np.loadtxt("test.dat")
print(train_data.shape)
print(test_data.shape)

(60000, 73)
(40000, 73)


In [13]:
train_data[1]

array([0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1.])

In [85]:
X = train_data[:,0:64]
muxinput1 = train_data[:,64:68]
muxinput2 = train_data[:,68:72]
y = train_data[:,72]
print(X.shape)

(60000, 64)


In [45]:

def compare1(a,b):
    n1 = 1*a[0] + 2*a[1] + 4*a[2] + 8*a[3]
    n2 = 1*b[0] + 2*b[1] + 4*b[2] + 8*b[3]

    res = 1 if n1 > n2 else 0
    return (n1, n2, res)

def compare2(a, b):
    x_0 = a[0]*b[0] + (1-a[0])*(1-b[0])
    x_1 = a[1]*b[1] + (1-a[1])*(1-b[1])
    x_2 = a[2]*b[2] + (1-a[2])*(1-b[2])
    x_3 = a[3]*b[3] + (1-a[3])*(1-b[3])

    greater = 1 if (a[3]*(1-b[3]) + x_3*a[2]*(1-b[2]) + x_3*x_2*a[1]*(1-b[1]) + x_3*x_2*x_1*a[0]*(1-b[0])) else 0
    return greater
    
s1 = time.perf_counter()
print(compare1([0,0,1,1],[1,0,1,1]))
e1 = time.perf_counter()
print("time: {} seconds".format(e1-s1))
s2 = time.perf_counter()
print(compare2([0,0,1,1],[1,0,1,1]))
e2 = time.perf_counter()
print("time: {} seconds".format(e2-s2))


(12, 13, 0)
time: 0.00020080000012967503 seconds
0
time: 0.00011769999991884106 seconds


In [83]:
model = LogisticRegression(C = 1000, fit_intercept = True, solver="newton-cg", verbose = False) 
groups = defaultdict(list)
outputs = defaultdict(list)
models = defaultdict(list)
labels = []
data = []

st = time.time()

for e in train_data:
    firstSbits = e[64:68]
    lastSbits = e[68:72]
    labels.append(e[72])
    data.append(e[0:64])
    i, j, flag = compare1(firstSbits, lastSbits)
    m1 = min(int(i), int(j))
    m2 = max(int(i), int(j))
    groups["{}->{}".format(m1, m2)].append(e[0:64])
    outputs["{}->{}".format(m1, m2)].append(e[72])
    models["{}->{}".format(m1, m2)] = model
    
for key in groups.keys():
    i = np.array(groups[key])
    o = np.array(outputs[key])
    models[key].fit(i,o)
    print(models[key].score(i,o))

et = time.time()
print("time: {} seconds".format(et-st))


0.6392156862745098
0.6746031746031746
0.6415094339622641
0.6287425149700598
0.654368932038835
0.6646216768916156
0.6385068762278978
0.6673866090712743
0.662
0.6626506024096386
0.6550308008213552
0.6455445544554456
0.6292585170340681
0.6633064516129032
0.6646090534979424
0.6421471172962226
0.6489151873767258
0.6174757281553398
0.6633266533066132
0.6408163265306123
0.6129707112970711
0.6767241379310345
0.6388888888888888
0.6340508806262231
0.6293436293436293
0.6612244897959184
0.64
0.6544715447154471
0.6268939393939394
0.630648330058939
0.6522633744855967
0.6343692870201096
0.6666666666666666
0.6588693957115009
0.6626984126984127
0.6477732793522267
0.6413662239089184
0.6816326530612244
0.6348314606741573
0.6458333333333334
0.6288032454361054
0.6194174757281553
0.6633663366336634
0.6300813008130082
0.6545454545454545
0.6475409836065574
0.6765327695560254
0.6149532710280374
0.6539923954372624
0.6530612244897959
0.615678776290631
0.6025369978858351
0.6379647749510763
0.6673228346456693
0.64

In [71]:
models["1->2"].get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': False,
 'warm_start': False}

In [84]:
st = time.time()

testgroups = defaultdict(list)
testoutputs = defaultdict(list)

for e in test_data:
    firstSbits = e[64:68]
    lastSbits = e[68:72]
    labels.append(e[72])
    data.append(e[0:64])
    i, j, flag = compare1(firstSbits, lastSbits)
    m1 = min(int(i), int(j))
    m2 = max(int(i), int(j))
    testgroups["{}->{}".format(m1, m2)].append(e[0:64])
    testoutputs["{}->{}".format(m1, m2)].append(e[72])

for key in groups.keys():
    i = np.array(testgroups[key])
    o = np.array(testoutputs[key])
    # models[key].fit(i,o)
    print(models[key].score(i,o))
    

0.5427728613569321
0.49014084507042255
0.5072886297376094
0.49700598802395207
0.5086705202312138
0.4921135646687697
0.49201277955271566
0.5068493150684932
0.49570200573065903
0.4672897196261682
0.4940119760479042
0.5
0.4984520123839009
0.5156695156695157
0.5286624203821656
0.5030674846625767
0.4954128440366973
0.47468354430379744
0.5432098765432098
0.5275080906148867
0.49732620320855614
0.4350453172205438
0.5046153846153846
0.4702194357366771
0.4389438943894389
0.5283582089552239
0.5290697674418605
0.4645161290322581
0.5366568914956011
0.49244712990936557
0.49216300940438873
0.5075528700906344
0.5174418604651163
0.47277936962750716
0.525
0.48534201954397393
0.49560117302052786
0.5274390243902439
0.5029411764705882
0.49318801089918257
0.47619047619047616
0.43730886850152906
0.47988505747126436
0.4691011235955056
0.5521472392638037
0.47041420118343197
0.4539877300613497
0.5073746312684366
0.5245398773006135
0.5177111716621253
0.49085365853658536
0.47368421052631576
0.5440729483282675
0.4